In [1]:
# === NOTEBOOK: PRE-PROCESSING (v7) - Odtworzenie logiki 'usecols' dla wszystkich potrzebnych danych ===

import pandas as pd

print("--- START: Tworzenie czystego pliku danych metodą wielokrotnego wczytywania 'usecols' ---")

# --- KROK 1: Definicja potrzebnych kolumn i wczytanie słownika ---
# Definiujemy indeksy kolumn, które chcemy "wyciąć" z brudnych plików
COLUMNS_TO_EXTRACT = {
    3: 'Title', 4: 'Description', 5: 'Area', 6: 'Price',
    12: 'BuiltYear', 17: 'NumberOfRooms', 35: 'Floor', 36: 'Floors'
}

# Wczytujemy słownik, który jest naszą mapą
try:
    df_slownik = pd.read_csv('slownik_finalny_z_hierarchia_polska.csv', sep=';')
    df_slownik['UlicaID'] = pd.to_numeric(df_slownik['UlicaID'], errors='coerce')
    df_slownik.dropna(subset=['UlicaID'], inplace=True)
    df_slownik['UlicaID'] = df_slownik['UlicaID'].astype(int)
    print(f"Słownik wczytany. Mamy {len(df_slownik)} unikalnych wpisów.")
except FileNotFoundError as e:
    raise

# --- KROK 2: Funkcja do wczytywania danych metodą 'usecols' ---
def extract_data_with_usecols(filepath, ulica_id_col_idx):
    print(f"\nEkstrakcja danych z pliku: {filepath}...")
    try:
        # Lista indeksów do wczytania: klucze ze słownika + UlicaID
        indices = list(COLUMNS_TO_EXTRACT.keys()) + [ulica_id_col_idx]
        
        # Wczytujemy tylko te kolumny, ignorując błędy strukturalne w innych
        df = pd.read_csv(
            filepath, sep=',', header=None, on_bad_lines='skip',
            usecols=indices, low_memory=False, dtype=str
        )
        
        # Zmieniamy nazwy kolumn
        rename_map = COLUMNS_TO_EXTRACT.copy()
        rename_map[ulica_id_col_idx] = 'UlicaID'
        df.rename(columns=rename_map, inplace=True)
        
        print(f"Udało się wyekstrahować {len(df)} wierszy.")
        return df
    except FileNotFoundError:
        print(f"OSTRZEŻENIE: Nie znaleziono pliku {filepath}. Pomijam.")
        return pd.DataFrame()
    except Exception as e:
        print(f"Błąd podczas ekstrakcji: {e}")
        return pd.DataFrame()

# --- KROK 3: Ekstrakcja i łączenie danych z obu plików ---
# Ustalamy indeks UlicaID
try:
    first_row = pd.read_csv('saleflats_2024_dateAdded_polska.csv', sep=',', header=None, on_bad_lines='skip', nrows=1)
    ulica_id_idx = first_row.shape[1] - 1
except Exception:
    print("Nie udało się odczytać struktury pliku, zakładam 106 kolumn (indeks UlicaID 105)")
    ulica_id_idx = 105 # Ostatnia kolumna z 106 to indeks 105

FILE_NEW = 'saleflats_2024_newestDate_polska.csv'
FILE_OLD = 'saleflats_2024_dateAdded_polska.csv'

df_newest = extract_data_with_usecols(FILE_NEW, ulica_id_idx)
df_added = extract_data_with_usecols(FILE_OLD, ulica_id_idx)

# Łączymy i deduplikujemy
df_combined_offers = pd.concat([df_newest, df_added])
df_combined_offers['UlicaID'] = pd.to_numeric(df_combined_offers['UlicaID'], errors='coerce')
df_combined_offers.dropna(subset=['UlicaID'], inplace=True)
df_combined_offers['UlicaID'] = df_combined_offers['UlicaID'].astype(int)
df_clean_offers = df_combined_offers.drop_duplicates(subset=['UlicaID'], keep='first')
print(f"\nPołączono i uzyskano {len(df_clean_offers)} unikalnych wierszy z ofert.")

# --- KROK 4: Ostateczne połączenie ze słownikiem ---
print("\nOstateczne łączenie ze słownikiem...")
df_final = pd.merge(df_slownik, df_clean_offers, on='UlicaID', how='inner')
print(f"Połączono ze słownikiem. Finalna liczba wierszy: {len(df_final)}")

# --- KROK 5: Zapisanie finalnego, czystego pliku ---
if not df_final.empty:
    OUTPUT_FILE = 'polska_flats_clean.csv'
    print(f"\nZapisywanie czystych danych do pliku: {OUTPUT_FILE}...")
    df_final.to_csv(OUTPUT_FILE, index=False, sep=';')
    print("Zapis zakończony sukcesem!")
    print("\nPróbka danych z czystego pliku:")
    display(df_final.head())
else:
    print("\nNadal nie udało się połączyć danych. Niezgodność jest fundamentalna.")

--- START: Tworzenie czystego pliku danych metodą wielokrotnego wczytywania 'usecols' ---
Słownik wczytany. Mamy 1896 unikalnych wpisów.

Ekstrakcja danych z pliku: saleflats_2024_newestDate_polska.csv...
Udało się wyekstrahować 755679 wierszy.

Ekstrakcja danych z pliku: saleflats_2024_dateAdded_polska.csv...
Udało się wyekstrahować 793664 wierszy.

Połączono i uzyskano 3114 unikalnych wierszy z ofert.

Ostateczne łączenie ze słownikiem...
Połączono ze słownikiem. Finalna liczba wierszy: 1896

Zapisywanie czystych danych do pliku: polska_flats_clean.csv...
Zapis zakończony sukcesem!

Próbka danych z czystego pliku:


,UlicaID,Ulica_Name,Dzielnica_Name,Miasto_Name,Title,Description,Area,Price,BuiltYear,NumberOfRooms,Floor,Floors
0,511,Adamów,Brak,Adamów,"Archicom Awicenny 3 pokoje 50,6m2 garaż podziemny",AGENT PROWADZĄCY: MACIEJ OPUCHLIK Tel: 602 101...,000zł Miejsce parkingowe na poziomie -1 - 30,000zł Czynsz administracyjny - 700zł / mc Reko...,14426.88,NaN,1,0
1,1911,Aleksandrów łódzki,Brak,Aleksandrów łódzki,"Mieszkanie, Warszawa, Śródmieście, 31 m²","Mieszkanie, Warszawa, Śródmieście, 31 m² Ogłos...",najbardziej luksusowe domy przed I wojną świa...,że fasada zdaje się falować,rue Hoża 39,Wiktoria Kawecka. L'immeuble a une élévation ...,73 m2,located in the central courtyard
2,17415,Brużyczka mała,Brużyczka mała,Aleksandrów łódzki,"Nowe Mieszkania 4 pokojowe, Rząska pod Krakowem",Diamentowe Zacisze Rząska to prestiżowe osiedl...,Lidl (5 minut samochodem),Biedronka (6 minut samochodem) Centrum outlet...,10869.55,NaN,1,0
3,24474,Karolew,Karolew,Aleksandrów łódzki,"Mieszkanie, Łódź, Górna, 52 m²","Mieszkanie, Łódź, Górna, 52 m² Ogłoszeniodawca...",jak i dojazd do ścisłego Centrum. Dostępny od...,00 pln Zapraszam na prezentacje ! DODATKOWE IN...,8730.77,NaN,0,0
4,8488,Jakubowice,Jakubowice,Annopol,"Piękne, urządzone, ul. Klaudyny, obok parku","Sprzedam bezpośrednio: Przestronne, znakomicie...",z własną Księgą Wieczystą sprzedaż bezpośrednia,"dziękuję uprzejmie pośrednikom""",17081.52,NaN,2,0
